In [1]:
import numpy as np
import os 
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torch import nn
import torchsummary
from torch.utils.data import DataLoader
from collections import defaultdict
from torchvision.utils import make_grid

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Utils

In [3]:
CLASS_NAME_TO_ID = {'BS': 0, 'SCRATCH': 1}
CLASS_ID_TO_NAME = {0: 'BS', 1: 'SCRATCH'}
BOX_COLOR = {'BS':(200, 0, 0), 'SCRATCH':(0, 0, 200)}
TEXT_COLOR = (255, 255, 255)

def save_model(model_state, model_name, save_dir="./trained_model"):
    os.makedirs(save_dir, exist_ok=True)
    torch.save(model_state, os.path.join(save_dir, model_name))


def visualize_bbox(image, bbox, class_name, color=BOX_COLOR, thickness=2):
    x_center, y_center, w, h = bbox
    x_min = int(x_center - w/2)
    y_min = int(y_center - h/2)
    x_max = int(x_center + w/2)
    y_max = int(y_center + h/2)
    
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=color[class_name], thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(image, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), color[class_name], -1)
    cv2.putText(
        image,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return image


def visualize(image, bboxes, category_ids):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
#         print('category_id: ',category_id)
        class_name = CLASS_ID_TO_NAME[category_id.item()]
        img = visualize_bbox(img, bbox, class_name)
    return img

# Datasets

In [4]:
class PET_dataset():
    def __init__(self,part,neck_dir,body_dir,phase, transformer=None, aug=None, aug_factor=0):
        self.neck_dir=neck_dir
        self.body_dir=body_dir
        self.part=part
        self.phase=phase
        self.transformer=transformer
        self.aug=aug
        self.aug_factor=aug_factor
        if(self.part=="body"):
            self.image_files = sorted([fn for fn in os.listdir(self.body_dir+"/"+self.phase+"/image") if fn.endswith("jpg")])
            self.label_files= sorted([lab for lab in os.listdir(self.body_dir+"/"+self.phase+"/label") if lab.endswith("txt")])
        elif(self.part=="neck"):
            self.image_files = sorted([fn for fn in os.listdir(self.neck_dir+"/"+self.phase+"/image") if fn.endswith("jpg")])
            self.label_files= sorted([lab for lab in os.listdir(self.neck_dir+"/"+self.phase+"/label") if lab.endswith("txt")])
        
        self.auged_img_list, self.auged_label_list=self.make_aug_list(self.image_files, self.label_files)
        
    def __getitem__(self,index):
        if(self.aug==None):
            filename, image = self.get_image(self.part, index)
            bboxes, class_ids = self.get_label(self.part, index)

            if(self.transformer):
                transformed_data=self.transformer(image=image, bboxes=bboxes, class_ids=class_ids)
                image = transformed_data['image']
                bboxes = np.array(transformed_data['bboxes'])
                class_ids = np.array(transformed_data['class_ids'])


            target = {}
    #         print(f'bboxes:{bboxes}\nclass_ids:{class_ids}\nlen_bboxes:{len(bboxes)}\nlen_class_ids:{len(class_ids)}')
    #         print(f'filename: {filename}')
            target["boxes"] = torch.Tensor(bboxes).float()
            target["labels"] = torch.Tensor(class_ids).long()

            ###
            bboxes=torch.Tensor(bboxes).float()
            class_ids=torch.Tensor(class_ids).long()
            target = np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1)
            ###
        else:
            image=self.auged_img_list[index][1]
            target=self.auged_label_list[index]
            filename=self.auged_img_list[index][0]
        return image, target, filename
    
    def __len__(self, ):
        length=0
        if(self.aug==None):
            length=len(self.image_files)
        else:
            length=len(self.auged_img_list)
        return length
    
    def make_aug_list(self,ori_image_list,ori_label_files):
        aug_image_list=[]
        aug_label_list=[]
        
        print(f"start making augmented images-- augmented factor:{self.aug_factor}")
        for i in range(len(ori_image_list)):
            filename, ori_image = self.get_image(self.part, i)
            ori_bboxes, ori_class_ids = self.get_label(self.part, i)
            for j in range(self.aug_factor):
                auged_data=self.aug(image=ori_image, bboxes=ori_bboxes, class_ids=ori_class_ids)
                image = auged_data['image']
                bboxes = np.array(auged_data['bboxes'])
                class_ids = np.array(auged_data['class_ids'])
                
                bboxes=torch.Tensor(bboxes).float()
                class_ids=torch.Tensor(class_ids).long()
                
                aug_image_list.append((filename, image))
                aug_label_list.append(np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1))
        
        print(f"total length of augmented images: {len(aug_image_list)}")
        
        return aug_image_list, aug_label_list
        
    
    def get_image(self, part, index): # 이미지 불러오는 함수
        filename = self.image_files[index]
        if(part=="body"):
#             print(f"body called!-> {self.part}")
            image_path = self.body_dir+"/"+self.phase+"/image/"+filename
        elif(part=="neck"):
#             print(f"neck called!-> {self.part}")
            image_path = self.neck_dir+"/"+self.phase+"/image/"+filename
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return filename, image
    
    def get_label(self, part, index): # label (box좌표, class_id) 불러오는 함수
        label_filename=self.label_files[index]
        if(part=="body"):
#             print(f"body label called!-> {self.part}")
            label_path = self.body_dir+"/"+self.phase+"/label/"+label_filename
        elif(part=="neck"):
#             print(f"neck label called!-> {self.part}")
            label_path = self.neck_dir+"/"+self.phase+"/label/"+label_filename
        with open(label_path, 'r') as file:
            labels = file.readlines()
        
        class_ids=[]
        bboxes=[]
        for label in labels:
            label=label.replace("\n", "")
            obj=label.split(' ')[0]
            coor=label.split(' ')[1:]
            obj=int(obj)
            coor=list(map(float, coor))
            class_ids.append(obj)
            bboxes.append(coor)
            
        return bboxes, class_ids
    

In [5]:
IMAGE_SIZE = 448

transformer = A.Compose([ 
        # bounding box의 변환, augmentation에서 albumentations는 Detection 학습을 할 때 굉장히 유용하다. 
        A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
        A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        # albumentations 라이브러리에서는 Normalization을 먼저 진행해 주고 tensor화를 진행해 주어야한다.
    ],
    # box 위치에 대한 transformation도 함께 진행된다. 
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
)

augmentator=A.Compose([
#     A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
    A.HorizontalFlip(p=0.7),
#     A.Sharpen(p=0.7),
    A.BBoxSafeRandomCrop(p=0.6),
    A.VerticalFlip (p=0.5),
    A.HueSaturationValue(p=0.5),
    A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
)

def collate_fn(batch):
    image_list = []
    target_list = []
    filename_list = []
    
    for a,b,c in batch:
        image_list.append(a)
        target_list.append(b)
        filename_list.append(c)

    return torch.stack(image_list, dim=0), target_list, filename_list


In [6]:
NECK_PATH = '/home/host_data/PET_data/Neck'
BODY_PATH = '/home/host_data/PET_data/Body'
trainset_yes_aug=PET_dataset(part='body',neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='train', transformer=transformer, aug=augmentator, aug_factor=50)
trainset_no_aug=PET_dataset(part='body',neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='train', transformer=transformer, aug=None)


start making augmented images-- augmented factor:50
total length of augmented images: 5850
start making augmented images-- augmented factor:0
total length of augmented images: 0


In [6]:
len(trainset_yes_aug)

NameError: name 'trainset_yes_aug' is not defined

In [7]:
@interact(index=(0, len(trainset_no_aug)-1))

def show_sample(index=0):
    image, target, filename = trainset_no_aug[index]
    image=image.permute(1,2,0).numpy()
    img_H, img_W, _ = image.shape
    print(filename)
    print(image.shape)
#     print(image)

#     bboxes = target['boxes']
#     class_ids = target["labels"]
    
    ###
    bboxes = target[:, 0:4]
    class_ids = target[:, 4]
    ###
    bboxes[:, [0,2]] *= img_W
    bboxes[:, [1,3]] *= img_H

    canvas = visualize(image, bboxes, class_ids)
    plt.figure(figsize=(6,6))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

# show_sample()

NameError: name 'trainset_no_aug' is not defined

In [8]:
@interact(index=(0, len(trainset_yes_aug)-1))

def show_sample(index=0):
    image, target, filename = trainset_yes_aug[index]
    image=image.permute(1,2,0).numpy()
    img_H, img_W, _ = image.shape
    print(filename)
    print(image.shape)
#     print(image)

#     bboxes = target['boxes']
#     class_ids = target["labels"]
    ###
    bboxes = target[:, 0:4]
    class_ids = target[:, 4]
    ###
    bboxes[:, [0,2]] *= img_W
    bboxes[:, [1,3]] *= img_H
    print(bboxes)

    canvas = visualize(image, bboxes, class_ids)
    plt.figure(figsize=(6,6))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

# show_sample()

NameError: name 'trainset_yes_aug' is not defined

## Model

In [6]:
class YOLO_SWIN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.num_classes = num_classes
        self.num_bboxes = 2
        self.grid_size = 7

#         resnet18 = torchvision.models.resnet18(pretrained = True)
        swin=torchvision.models.swin_v2_t(weights='IMAGENET1K_V1')
        layers = [m for m in swin.children()] #Resnet에서 Yolo에서 가져올수 있을만한 layer만 선별적으로 가져오기 위해서

        # 기존 Resnet18의 layer들중에서 맨 뒤에 두개만 제외하고 다 가져와서 Backbone으로 사용
        self.backbone = nn.Sequential(*layers[:-3]) 
        self.head = nn.Sequential(
                nn.Conv2d(in_channels=768, out_channels=1024, kernel_size=1, padding=0,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1,bias=False),
                nn.BatchNorm2d(1024),
                nn.ReLU(inplace=True),

                nn.Conv2d(in_channels=1024, out_channels=(4+1)*self.num_bboxes+num_classes, kernel_size=1, padding=0, bias=False),
                nn.AdaptiveAvgPool2d(output_size=(self.grid_size, self.grid_size))
            )

    def forward(self, x):
        out = self.backbone(x)
        # out = self.neck(out)
        out = self.head(out) # input (batch, 3, 448, 448) -> output feature (batch, 12, 7, 7)
        return out


In [7]:
NUM_CLASSES = 2
model = YOLO_SWIN(num_classes=NUM_CLASSES)
model.to(device)

YOLO_SWIN(
  (backbone): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): Permute()
        (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (1): Sequential(
        (0): SwinTransformerBlockV2(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): ShiftedWindowAttentionV2(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (0): Linear(in_features=96, out_f

In [8]:
torchsummary.summary(model, (3,448,448))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 112, 112]           4,704
           Permute-2         [-1, 112, 112, 96]               0
         LayerNorm-3         [-1, 112, 112, 96]             192
            Linear-4          [-1, 15, 15, 512]           1,536
              ReLU-5          [-1, 15, 15, 512]               0
            Linear-6            [-1, 15, 15, 3]           1,536
ShiftedWindowAttentionV2-7         [-1, 112, 112, 96]               0
         LayerNorm-8         [-1, 112, 112, 96]             192
   StochasticDepth-9         [-1, 112, 112, 96]               0
           Linear-10        [-1, 112, 112, 384]          37,248
             GELU-11        [-1, 112, 112, 384]               0
          Dropout-12        [-1, 112, 112, 384]               0
           Linear-13         [-1, 112, 112, 96]          36,960
          Dropout-14         [-1,

In [9]:
x = torch.randn(1, 3, 448, 448).to(device)
with torch.no_grad():
    y = model(x)
print(y.shape)

torch.Size([1, 12, 7, 7])


# Loss func

In [10]:
class YOLO_LOSS():
    def __init__(self, num_classes, device, lambda_coord=5., lambda_noobj=0.5):
        self.num_classes = num_classes
        self.device = device
        self.grid_size = 7
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj
        self.mse_loss = nn.MSELoss(reduction="sum")

    def __call__(self, predictions, targets):
        self.batch_size, _, _, _ = predictions.shape
        groundtruths = self.build_batch_target_grid(targets)
        groundtruths = groundtruths.to(self.device)
        
        with torch.no_grad():
            iou1 = self.get_IoU(predictions[:, 1:5, ...], groundtruths[:, 1:5, ...])
            iou2 = self.get_IoU(predictions[:, 6:10, ...], groundtruths[:, 1:5, ...])

        ious = torch.stack([iou1, iou2], dim=1)
        max_iou, best_box = ious.max(dim=1, keepdim=True)
        max_iou = torch.cat([max_iou, max_iou], dim=1)
        best_box = torch.cat([best_box.eq(0), best_box.eq(1)], dim=1)

        predictions_ = predictions[:, :5*2, ...].reshape(self.batch_size, 2, 5, self.grid_size, self.grid_size)
        obj_pred = predictions_[:, :, 0, ...]
        xy_pred = predictions_[:, :, 1:3, ...]
        wh_pred = predictions_[:, :, 3:5, ...]
        cls_pred = predictions[:, 5*2:, ...]

        groundtruths_ = groundtruths[:, :5, ...].reshape(self.batch_size, 1, 5, self.grid_size, self.grid_size)
        obj_target = groundtruths_[:, :, 0, ...]
        xy_target = groundtruths_[:, :, 1:3, ...]
        wh_target= groundtruths_[:, :, 3:5, ...]
        cls_target = groundtruths[:, 5:, ...]
        
        positive = obj_target * best_box

        obj_loss = self.mse_loss(positive * obj_pred, positive * ious)
        noobj_loss = self.mse_loss((1 - positive) * obj_pred, ious*0)
        xy_loss = self.mse_loss(positive.unsqueeze(dim=2) * xy_pred, positive.unsqueeze(dim=2) * xy_target)
        wh_loss = self.mse_loss(positive.unsqueeze(dim=2) * (wh_pred.sign() * (wh_pred.abs() + 1e-8).sqrt()),
                           positive.unsqueeze(dim=2) * (wh_target + 1e-8).sqrt())
        cls_loss = self.mse_loss(obj_target * cls_pred, cls_target)
        
        obj_loss /= self.batch_size
        noobj_loss /= self.batch_size
        bbox_loss = (xy_loss+wh_loss) / self.batch_size
        cls_loss /= self.batch_size
        
        total_loss = obj_loss + self.lambda_noobj*noobj_loss + self.lambda_coord*bbox_loss + cls_loss
        return total_loss, (obj_loss.item(), noobj_loss.item(), bbox_loss.item(), cls_loss.item())
    
    def build_target_grid(self, target):
        target_grid = torch.zeros((1+4+self.num_classes, self.grid_size, self.grid_size), device=self.device)

        for gt in target:
            xc, yc, w, h, cls_id = gt
            xn = (xc % (1/self.grid_size))
            yn = (yc % (1/self.grid_size))
            cls_id = int(cls_id)

            i_grid = int(xc * self.grid_size)
            j_grid = int(yc * self.grid_size)
            target_grid[0, j_grid, i_grid] = 1
            target_grid[1:5, j_grid, i_grid] = torch.Tensor([xn,yn,w,h])
#             print(5+cls_id, j_grid, i_grid)
            target_grid[5+cls_id, j_grid, i_grid] = 1

        return target_grid
    
    def build_batch_target_grid(self, targets):
        target_grid_batch = torch.stack([self.build_target_grid(target) for target in targets], dim=0)
        return target_grid_batch
    
    def get_IoU(self, cbox1, cbox2):
        box1 = self.xywh_to_xyxy(cbox1)
        box2 = self.xywh_to_xyxy(cbox2)

        x1 = torch.max(box1[:, 0, ...], box2[:, 0, ...])
        y1 = torch.max(box1[:, 1, ...], box2[:, 1, ...])
        x2 = torch.min(box1[:, 2, ...], box2[:, 2, ...])
        y2 = torch.min(box1[:, 3, ...], box2[:, 3, ...])

        intersection = (x2-x1).clamp(min=0) * (y2-y1).clamp(min=0)
        union = abs(cbox1[:, 2, ...]*cbox1[:, 3, ...]) + \
                abs(cbox2[:, 2, ...]*cbox2[:, 3, ...]) - intersection

        intersection[intersection.gt(0)] = intersection[intersection.gt(0)] / union[intersection.gt(0)]
        return intersection
    
    def generate_xy_normed_grid(self):
        y_offset, x_offset = torch.meshgrid(torch.arange(self.grid_size), torch.arange(self.grid_size))
        xy_grid = torch.stack([x_offset, y_offset], dim=0)
        xy_normed_grid = xy_grid / self.grid_size
        return xy_normed_grid.to(self.device)

    def xywh_to_xyxy(self, bboxes):
        xy_normed_grid = self.generate_xy_normed_grid()
        xcyc = bboxes[:,0:2,...] + xy_normed_grid.tile(self.batch_size, 1,1,1)
        wh = bboxes[:,2:4,...]
        x1y1 = xcyc - (wh/2)
        x2y2 = xcyc + (wh/2)
        return torch.cat([x1y1, x2y2], dim=1)

# Train

In [11]:
def train_one_epoch(dataloaders, model, criterion, optimizer, device):
    train_loss = defaultdict(float)
    val_loss = defaultdict(float)
    
    for phase in ["train", "val"]:
        if phase == "train":
            model.train()
        else:
            model.eval()
        
        running_loss = defaultdict(float)
        for index, batch in enumerate(dataloaders[phase]):
            images = batch[0].to(device)
            targets = batch[1]
            filenames = batch[2]
            
            with torch.set_grad_enabled(phase == "train"): # phase가 train 일때만 gradient 추적기능을 킨다.
                predictions = model(images) #prediction shape=> B,12,7,7
#             print(f"predictions:{predictions}, \ntargets: {targets}\n")
            loss, (obj_loss, noobj_loss, bbox_loss, cls_loss) = criterion(predictions, targets)
#             print(f"loss:{loss}, obj_loss:{obj_loss}, noobj_loss:{noobj_loss}\nbbox_loss:{bbox_loss}, cls_loss:{cls_loss}\n--------------\n")
            if phase == "train":
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # 현재 epoch단계에서 loss가 얼마인지 running loss 가출력
                running_loss["total_loss"] += loss.item()
                running_loss["obj_loss"] += obj_loss
                running_loss["noobj_loss"] += noobj_loss
                running_loss["bbox_loss"] += bbox_loss
                running_loss["cls_loss"] += cls_loss
                
                train_loss["total_loss"] += loss.item()
                train_loss["obj_loss"] += obj_loss
                train_loss["noobj_loss"] += noobj_loss
                train_loss["bbox_loss"] += bbox_loss
                train_loss["cls_loss"] += cls_loss
                
                if (index > 0) and (index % VERBOSE_FREQ) == 0:
                    text = f"<<<iteration:[{index}/{len(dataloaders[phase])}] - "
                    for k, v in running_loss.items():
                        text += f"{k}: {v/VERBOSE_FREQ:.4f}  "
                        running_loss[k] = 0.
                    print(text)
            else:
                val_loss["total_loss"] += loss.item()
                val_loss["obj_loss"] += obj_loss
                val_loss["noobj_loss"] += noobj_loss
                val_loss["bbox_loss"] += bbox_loss
                val_loss["cls_loss"] += cls_loss

    for k in train_loss.keys():
        train_loss[k] /= len(dataloaders["train"])
        val_loss[k] /= len(dataloaders["val"])
    return train_loss, val_loss

In [12]:
def build_dataloader(part, NECK_PATH, BODY_PATH, batch_size=2, aug_factor=0):
    IMAGE_SIZE = 448
    transformer = A.Compose([
            A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
            A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
    )
    augmentator=A.Compose([
    #     A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
        A.HorizontalFlip(p=0.7),
    #     A.Sharpen(p=0.7),
        A.BBoxSafeRandomCrop(p=0.6),
        A.VerticalFlip (p=0.6),
        A.HueSaturationValue(p=0.6),
        A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
        A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
    )
    
    dataloaders = {}
#     train_dataset = Detection_dataset(data_dir=data_dir, phase="train", transformer=transformer)
    train_dataset=PET_dataset(part ,neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='train', transformer=transformer, aug=augmentator, aug_factor=aug_factor)
    dataloaders["train"] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

#     val_dataset = Detection_dataset(data_dir=data_dir, phase="val", transformer=transformer)
    val_dataset=PET_dataset(part ,neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='valid', transformer=transformer, aug=augmentator, aug_factor=aug_factor)
    dataloaders["val"] = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return dataloaders

In [13]:
# data_dir = "/content/drive/MyDrive/fastCamMedicalProj/DATASET/DATASET/Detection/"
NECK_PATH = '/home/host_data/PET_data/Neck'
BODY_PATH = '/home/host_data/PET_data/Body'
is_cuda = True

NUM_CLASSES = 2
IMAGE_SIZE = 448
BATCH_SIZE = 4
VERBOSE_FREQ = 20
LR=0.0001
AUG_FACTOR=30
BACKBONE="YOLO_SWIN_T"
PART="body"
num_epochs = 100
# DEVICE = torch.device('cuda' if torch.cuda.is_available and is_cuda else 'cpu')

dataloaders = build_dataloader(part=PART,NECK_PATH=NECK_PATH,BODY_PATH=BODY_PATH,batch_size=BATCH_SIZE, aug_factor=AUG_FACTOR)
model = YOLO_SWIN(num_classes=NUM_CLASSES)
model = model.to(device)
criterion = YOLO_LOSS(num_classes=NUM_CLASSES, device=device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

start making augmented images-- augmented factor:30
total length of augmented images: 3510
start making augmented images-- augmented factor:30
total length of augmented images: 600


In [14]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="yolo_swin",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "batch_size": BATCH_SIZE,
    "architecture": BACKBONE,
    "dataset": "BODY",
    "epochs": num_epochs,
    "aug factor":AUG_FACTOR,
    }
)

wandb: Currently logged in as: gomduribo (urp). Use `wandb login --relogin` to force relogin


In [15]:
best_epoch = 0
best_score = float('inf')
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    train_loss, val_loss = train_one_epoch(dataloaders, model, criterion, optimizer, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
#     train_loss["obj_loss"] += obj_loss
#     train_loss["noobj_loss"] += noobj_loss
#     train_loss["bbox_loss"] += bbox_loss
#     train_loss["cls_loss"] += cls_loss
    wandb.log({"Train Loss": train_loss['total_loss'],
               "Train obj Loss":train_loss["obj_loss"],
               "Train bbox Loss":train_loss["bbox_loss"],
               "Train class Loss":train_loss["cls_loss"],
               "Val Loss": val_loss['total_loss'],
               "Val obj Loss":val_loss["obj_loss"],
               "Val bbox Loss":val_loss["bbox_loss"],
               "Val class Loss":val_loss["cls_loss"],})
    print(f"\nepoch:{epoch+1}/{num_epochs} - Train Loss: {train_loss['total_loss']:.4f}, Val Loss: {val_loss['total_loss']:.4f}\n")
    
    if (epoch+1) % 10 == 0:
        save_model(model.state_dict(), f'model_{epoch+1}.pth', save_dir=f"./trained_model/{BACKBONE}_{PART}_LR{LR}_AUG{AUG_FACTOR}")
wandb.finish()

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3423.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<<<iteration:[20/878] - total_loss: 24.1707  obj_loss: 0.1836  noobj_loss: 3.4645  bbox_loss: 3.6269  cls_loss: 4.1205  
<<<iteration:[40/878] - total_loss: 16.7985  obj_loss: 0.1470  noobj_loss: 2.5177  bbox_loss: 2.4995  cls_loss: 2.8952  
<<<iteration:[60/878] - total_loss: 14.8321  obj_loss: 0.1544  noobj_loss: 2.6462  bbox_loss: 2.0846  cls_loss: 2.9315  
<<<iteration:[80/878] - total_loss: 20.4857  obj_loss: 0.3012  noobj_loss: 4.6454  bbox_loss: 2.9843  cls_loss: 2.9404  
<<<iteration:[100/878] - total_loss: 16.4303  obj_loss: 0.0941  noobj_loss: 1.9319  bbox_loss: 2.5196  cls_loss: 2.7722  
<<<iteration:[120/878] - total_loss: 15.8625  obj_loss: 0.0932  noobj_loss: 1.8765  bbox_loss: 2.4196  cls_loss: 2.7330  
<<<iteration:[140/878] - total_loss: 12.2725  obj_loss: 0.0835  noobj_loss: 1.6928  bbox_loss: 1.7151  cls_loss: 2.7671  
<<<iteration:[160/878] - total_loss: 13.7770  obj_loss: 0.0804  noobj_loss: 1.5613  bbox_loss: 2.0308  cls_loss: 2.7617  
<<<iteration:[180/878] - tot

<<<iteration:[520/878] - total_loss: 4.5186  obj_loss: 0.0709  noobj_loss: 0.0857  bbox_loss: 0.3615  cls_loss: 2.5976  
<<<iteration:[540/878] - total_loss: 5.5359  obj_loss: 0.0647  noobj_loss: 0.0618  bbox_loss: 0.5740  cls_loss: 2.5702  
<<<iteration:[560/878] - total_loss: 3.9063  obj_loss: 0.0808  noobj_loss: 0.0663  bbox_loss: 0.2300  cls_loss: 2.6423  
<<<iteration:[580/878] - total_loss: 3.8926  obj_loss: 0.0851  noobj_loss: 0.0559  bbox_loss: 0.2415  cls_loss: 2.5721  
<<<iteration:[600/878] - total_loss: 4.6235  obj_loss: 0.0626  noobj_loss: 0.0562  bbox_loss: 0.3890  cls_loss: 2.5877  
<<<iteration:[620/878] - total_loss: 4.8927  obj_loss: 0.0822  noobj_loss: 0.0664  bbox_loss: 0.3823  cls_loss: 2.8658  
<<<iteration:[640/878] - total_loss: 4.1133  obj_loss: 0.0752  noobj_loss: 0.0596  bbox_loss: 0.2753  cls_loss: 2.6319  
<<<iteration:[660/878] - total_loss: 3.4552  obj_loss: 0.0685  noobj_loss: 0.0646  bbox_loss: 0.1978  cls_loss: 2.3656  
<<<iteration:[680/878] - total_l

<<<iteration:[140/878] - total_loss: 6.7775  obj_loss: 0.0709  noobj_loss: 0.0465  bbox_loss: 0.8085  cls_loss: 2.6410  
<<<iteration:[160/878] - total_loss: 7.6416  obj_loss: 0.0902  noobj_loss: 0.0469  bbox_loss: 1.0277  cls_loss: 2.3894  
<<<iteration:[180/878] - total_loss: 5.8051  obj_loss: 0.0595  noobj_loss: 0.0483  bbox_loss: 0.6554  cls_loss: 2.4447  
<<<iteration:[200/878] - total_loss: 4.5505  obj_loss: 0.0868  noobj_loss: 0.0484  bbox_loss: 0.3884  cls_loss: 2.4974  
<<<iteration:[220/878] - total_loss: 7.0316  obj_loss: 0.0970  noobj_loss: 0.0555  bbox_loss: 0.8794  cls_loss: 2.5098  
<<<iteration:[240/878] - total_loss: 5.0737  obj_loss: 0.0538  noobj_loss: 0.0671  bbox_loss: 0.4746  cls_loss: 2.6134  
<<<iteration:[260/878] - total_loss: 3.7928  obj_loss: 0.0934  noobj_loss: 0.0457  bbox_loss: 0.2275  cls_loss: 2.5393  
<<<iteration:[280/878] - total_loss: 3.9719  obj_loss: 0.0987  noobj_loss: 0.0500  bbox_loss: 0.2408  cls_loss: 2.6443  
<<<iteration:[300/878] - total_l

<<<iteration:[640/878] - total_loss: 3.8921  obj_loss: 0.0941  noobj_loss: 0.0355  bbox_loss: 0.2504  cls_loss: 2.5283  
<<<iteration:[660/878] - total_loss: 3.7239  obj_loss: 0.0965  noobj_loss: 0.0395  bbox_loss: 0.2591  cls_loss: 2.3120  
<<<iteration:[680/878] - total_loss: 3.2416  obj_loss: 0.0790  noobj_loss: 0.0361  bbox_loss: 0.1185  cls_loss: 2.5517  
<<<iteration:[700/878] - total_loss: 3.5627  obj_loss: 0.0774  noobj_loss: 0.0396  bbox_loss: 0.2270  cls_loss: 2.3306  
<<<iteration:[720/878] - total_loss: 3.5397  obj_loss: 0.0693  noobj_loss: 0.0505  bbox_loss: 0.1817  cls_loss: 2.5367  
<<<iteration:[740/878] - total_loss: 3.3685  obj_loss: 0.0931  noobj_loss: 0.0652  bbox_loss: 0.1307  cls_loss: 2.5893  
<<<iteration:[760/878] - total_loss: 3.2793  obj_loss: 0.0917  noobj_loss: 0.0363  bbox_loss: 0.1412  cls_loss: 2.4635  
<<<iteration:[780/878] - total_loss: 3.7783  obj_loss: 0.0876  noobj_loss: 0.0432  bbox_loss: 0.2190  cls_loss: 2.5741  
<<<iteration:[800/878] - total_l

...

<<<iteration:[100/878] - total_loss: 0.5609  obj_loss: 0.2218  noobj_loss: 0.1196  bbox_loss: 0.0246  cls_loss: 0.1563  
<<<iteration:[120/878] - total_loss: 0.5086  obj_loss: 0.2306  noobj_loss: 0.1292  bbox_loss: 0.0272  cls_loss: 0.0774  
<<<iteration:[140/878] - total_loss: 0.6526  obj_loss: 0.2530  noobj_loss: 0.1328  bbox_loss: 0.0248  cls_loss: 0.2093  
<<<iteration:[160/878] - total_loss: 0.5826  obj_loss: 0.2424  noobj_loss: 0.1251  bbox_loss: 0.0293  cls_loss: 0.1313  
<<<iteration:[180/878] - total_loss: 0.6084  obj_loss: 0.2038  noobj_loss: 0.1370  bbox_loss: 0.0336  cls_loss: 0.1681  
<<<iteration:[200/878] - total_loss: 0.5426  obj_loss: 0.2237  noobj_loss: 0.1219  bbox_loss: 0.0236  cls_loss: 0.1400  
<<<iteration:[220/878] - total_loss: 0.5615  obj_loss: 0.2398  noobj_loss: 0.1416  bbox_loss: 0.0253  cls_loss: 0.1242  
<<<iteration:[240/878] - total_loss: 0.5134  obj_loss: 0.2097  noobj_loss: 0.1537  bbox_loss: 0.0283  cls_loss: 0.0856  
<<<iteration:[260/878] - total_l

<<<iteration:[600/878] - total_loss: 0.5566  obj_loss: 0.2819  noobj_loss: 0.1504  bbox_loss: 0.0222  cls_loss: 0.0886  
<<<iteration:[620/878] - total_loss: 0.5644  obj_loss: 0.2771  noobj_loss: 0.1355  bbox_loss: 0.0242  cls_loss: 0.0987  
<<<iteration:[640/878] - total_loss: 0.4689  obj_loss: 0.2173  noobj_loss: 0.1524  bbox_loss: 0.0230  cls_loss: 0.0606  
<<<iteration:[660/878] - total_loss: 0.5371  obj_loss: 0.2449  noobj_loss: 0.1282  bbox_loss: 0.0238  cls_loss: 0.1090  
<<<iteration:[680/878] - total_loss: 0.5292  obj_loss: 0.2287  noobj_loss: 0.1261  bbox_loss: 0.0204  cls_loss: 0.1354  
<<<iteration:[700/878] - total_loss: 0.6235  obj_loss: 0.2992  noobj_loss: 0.1564  bbox_loss: 0.0254  cls_loss: 0.1193  
<<<iteration:[720/878] - total_loss: 0.5227  obj_loss: 0.2397  noobj_loss: 0.1350  bbox_loss: 0.0249  cls_loss: 0.0912  
<<<iteration:[740/878] - total_loss: 0.5102  obj_loss: 0.2201  noobj_loss: 0.1488  bbox_loss: 0.0228  cls_loss: 0.1016  
<<<iteration:[760/878] - total_l

<<<iteration:[220/878] - total_loss: 0.5197  obj_loss: 0.2339  noobj_loss: 0.1340  bbox_loss: 0.0199  cls_loss: 0.1192  
<<<iteration:[240/878] - total_loss: 0.4970  obj_loss: 0.2419  noobj_loss: 0.1415  bbox_loss: 0.0215  cls_loss: 0.0767  
<<<iteration:[260/878] - total_loss: 0.5582  obj_loss: 0.2392  noobj_loss: 0.1478  bbox_loss: 0.0252  cls_loss: 0.1191  
<<<iteration:[280/878] - total_loss: 0.4709  obj_loss: 0.2348  noobj_loss: 0.1291  bbox_loss: 0.0208  cls_loss: 0.0676  
<<<iteration:[300/878] - total_loss: 0.5142  obj_loss: 0.2589  noobj_loss: 0.1270  bbox_loss: 0.0217  cls_loss: 0.0832  
<<<iteration:[320/878] - total_loss: 0.6501  obj_loss: 0.3021  noobj_loss: 0.1597  bbox_loss: 0.0255  cls_loss: 0.1407  
<<<iteration:[340/878] - total_loss: 0.4375  obj_loss: 0.1960  noobj_loss: 0.1292  bbox_loss: 0.0194  cls_loss: 0.0798  
<<<iteration:[360/878] - total_loss: 0.4901  obj_loss: 0.2586  noobj_loss: 0.1387  bbox_loss: 0.0218  cls_loss: 0.0533  
<<<iteration:[380/878] - total_l

<<<iteration:[720/878] - total_loss: 0.4881  obj_loss: 0.1939  noobj_loss: 0.1379  bbox_loss: 0.0237  cls_loss: 0.1068  
<<<iteration:[740/878] - total_loss: 0.5308  obj_loss: 0.2634  noobj_loss: 0.1324  bbox_loss: 0.0241  cls_loss: 0.0807  
<<<iteration:[760/878] - total_loss: 0.4838  obj_loss: 0.2405  noobj_loss: 0.1284  bbox_loss: 0.0179  cls_loss: 0.0899  
<<<iteration:[780/878] - total_loss: 0.4727  obj_loss: 0.2203  noobj_loss: 0.1289  bbox_loss: 0.0214  cls_loss: 0.0808  
<<<iteration:[800/878] - total_loss: 0.5770  obj_loss: 0.2873  noobj_loss: 0.1435  bbox_loss: 0.0232  cls_loss: 0.1020  
<<<iteration:[820/878] - total_loss: 0.6634  obj_loss: 0.2911  noobj_loss: 0.1611  bbox_loss: 0.0325  cls_loss: 0.1290  
<<<iteration:[840/878] - total_loss: 0.6063  obj_loss: 0.2878  noobj_loss: 0.1761  bbox_loss: 0.0273  cls_loss: 0.0940  
<<<iteration:[860/878] - total_loss: 0.6250  obj_loss: 0.2693  noobj_loss: 0.1356  bbox_loss: 0.0230  cls_loss: 0.1731  

epoch:99/100 - Train Loss: 0.54

Train Loss,█▃▄▄▃▄▃▃▃▃▃▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train bbox Loss,█▂▃▂▂▃▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train class Loss,██▇█▇█▇▇▇▇▇▇▇▇▆▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train obj Loss,▁▁▂▁▂▂▂▂▂▂▂▂▂▂▂▃▄▅▅▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇█▇█▇██
Val Loss,▅▃▆▃▃▃▃▃▃▄█▅▃▃▃▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Val bbox Loss,▅▂▅▂▂▁▁▁▁▃█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Val class Loss,████▇▇█▇▇▇▇▇▇█▇▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val obj Loss,▁▁▂▄▃▄▃▄▅▃▂▄▄▃▃▆▇▇▅▆▇▇▅▇▆▇▄▅▇▇█▇▇▅▆▇██▇▇
Train Loss,0.54603
Train bbox Loss,0.0238
Train class Loss,0.10737


# Test Dataset Inference

In [14]:
import numpy as np
import os 
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torch import nn
import torchsummary
from torch.utils.data import DataLoader
from collections import defaultdict
from torchvision.utils import make_grid

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
def load_model(ckpt_path, num_classes, device):
    checkpoint = torch.load(ckpt_path, map_location=device)
    model = YOLO_SWIN(num_classes=num_classes)
    model.load_state_dict(checkpoint)
    model = model.to(device)
    model.eval()
    return model

In [47]:
IMAGE_SIZE=448
transformer = A.Compose([
            A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
            A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
)

In [48]:
ckpt_path="./trained_model/YOLO_SWIN_T_body_LR0.0001_AUG30/model_90.pth"
model = load_model(ckpt_path, NUM_CLASSES, device)

In [49]:
test_dataset=PET_dataset("body" ,neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='test', transformer=transformer, aug=None)
test_dataloaders = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

start making augmented images-- augmented factor:0
total length of augmented images: 0


In [50]:
len(test_dataset)

20

In [51]:
@torch.no_grad()
def model_predict(image, model, conf_thres=0.2, iou_threshold=0.1):
    predictions = model(image)
    prediction = predictions.detach().cpu().squeeze(dim=0)
#     print(prediction.shape)
    
    grid_size = prediction.shape[-1]
    y_grid, x_grid = torch.meshgrid(torch.arange(grid_size), torch.arange(grid_size))
    stride_size = IMAGE_SIZE/grid_size

    conf = prediction[[0,5], ...].reshape(1, -1)
    xc = (prediction[[1,6], ...] * IMAGE_SIZE + x_grid*stride_size).reshape(1,-1)
    yc = (prediction[[2,7], ...] * IMAGE_SIZE + y_grid*stride_size).reshape(1,-1)
    w = (prediction[[3,8], ...] * IMAGE_SIZE).reshape(1,-1)
    h = (prediction[[4,9], ...] * IMAGE_SIZE).reshape(1,-1)
    cls = torch.max(prediction[10:, ...].reshape(NUM_CLASSES, -1), dim=0).indices.tile(1,2)
    
    x_min = xc - w/2
    y_min = yc - h/2
    x_max = xc + w/2
    y_max = yc + h/2

    prediction_res = torch.cat([x_min, y_min, x_max, y_max, conf, cls], dim=0)
    prediction_res = prediction_res.transpose(0,1)

    # x_min과 y_min이 음수가 되지않고, x_max와 y_max가 이미지 크기를 넘지 않게 제한
    prediction_res[:, 2].clip(min=0, max=image.shape[1]) 
    prediction_res[:, 3].clip(min=0, max=image.shape[0])
        
    pred_res = prediction_res[prediction_res[:, 4] > conf_thres]
    nms_index = torchvision.ops.nms(boxes=pred_res[:, 0:4], scores=pred_res[:, 4], iou_threshold=iou_threshold)
    pred_res_ = pred_res[nms_index].numpy()
    
    n_obj = pred_res_.shape[0]
    bboxes = np.zeros(shape=(n_obj, 4), dtype=np.float32)
    bboxes[:, 0:2] = (pred_res_[:, 0:2] + pred_res_[:, 2:4]) / 2
    bboxes[:, 2:4] = pred_res_[:, 2:4] - pred_res_[:, 0:2]
    scores = pred_res_[:, 4]
    class_ids = pred_res_[:, 5]
    
    # 이미지 값이 들어가면 모델을 통해서, 후처리까지 포함된 yolo 포멧의 box좌표, 그 좌표에 대한 confidence score
    # 그리고 class id를 반환
    return bboxes, scores, class_ids

In [54]:
pred_images = []
pred_labels =[]

for index, batch in enumerate(test_dataloaders):
    images = batch[0].to(device)
    bboxes, scores, class_ids = model_predict(images, model, conf_thres=0.2, iou_threshold=0.1)
    
    if len(bboxes) > 0:
        prediction_yolo = np.concatenate([bboxes, scores[:, np.newaxis], class_ids[:, np.newaxis]], axis=1)
    else:
        prediction_yolo = np.array([])
    
    # 텐서형의 이미지를 다시 unnormalize를 시키고, 다시 chw를 hwc로 바꾸고 넘파이로 바꾼다.
    np_image = make_grid(images[0], normalize=True).cpu().permute(1,2,0).numpy()
    pred_images.append(np_image)
    pred_labels.append(prediction_yolo)

    

In [55]:
from ipywidgets import interact

@interact(index=(0,len(pred_images)-1))
def show_result(index=0):
    print(pred_labels[index])
    if len(pred_labels[index]) > 0:
        result = visualize(pred_images[index], pred_labels[index][:, 0:4], pred_labels[index][:, 5])
    else:
        result = pred_images[index]
        
    plt.figure(figsize=(6,6))
    plt.imshow(result)
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=19), Output()), _dom_classes=('widget-intera…

# 탐지성능검증을 위한 지표 적용

In [56]:
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

NOTE! Installing ujson may make loading annotations faster.


In [57]:
annFile = "/home/host_data/PET_data/Body/test/test.json"
# val.json은 COCO api의 Ground truth input 값으로 들어갈 것이다.
with open(annFile, mode='r') as f:
    json_data = json.load(f)
imageToid = json_data["imageToid"]
# 데이터셋을 구축할 때, 이미지, target, filename을 반환하게 했는데, 그 filename을 통해서 id를 추적하고
# val.json에 있는 annotation값 즉 bbox좌표(ground truth)를 알아내서 IoU를 계산한다.
cocoGt=COCO(annFile)

Loading annotations into memory...
Done (t=0.00s)
Creating index...
index created!


In [58]:
json_data

{'images': [{'file_name': 'shape4_4.jpg',
   'height': 448,
   'width': 448,
   'id': 0},
  {'file_name': 'shape1_129.jpg', 'height': 448, 'width': 448, 'id': 1},
  {'file_name': 'shape1_17.jpg', 'height': 448, 'width': 448, 'id': 2},
  {'file_name': 'shape1_32.jpg', 'height': 448, 'width': 448, 'id': 3},
  {'file_name': 'shape1_53.jpg', 'height': 448, 'width': 448, 'id': 4},
  {'file_name': 'shape1_59.jpg', 'height': 448, 'width': 448, 'id': 5},
  {'file_name': 'shape1_67.jpg', 'height': 448, 'width': 448, 'id': 6},
  {'file_name': 'shape1_81.jpg', 'height': 448, 'width': 448, 'id': 7},
  {'file_name': 'shape1_93.jpg', 'height': 448, 'width': 448, 'id': 8},
  {'file_name': 'shape3_115.jpg', 'height': 448, 'width': 448, 'id': 9},
  {'file_name': 'shape3_131.jpg', 'height': 448, 'width': 448, 'id': 10},
  {'file_name': 'shape3_143.jpg', 'height': 448, 'width': 448, 'id': 11},
  {'file_name': 'shape3_40.jpg', 'height': 448, 'width': 448, 'id': 12},
  {'file_name': 'shape3_64.jpg', 'heigh

In [72]:
cocoGt.showAnns(anns='bbox')

Exception: datasetType not supported

In [59]:
# COCO에서는 x_min, y_min, w, h를 지원하기 때문에
# x_cen, y_cen, w, h를 x_min, y_min, w, h로 바꿔줘야한다.
def XcenYcenWH_to_XminYmin_WH(box):
    Xmin = box[:, 0] - box[:, 2]/2
    Ymin = box[:, 1] - box[:, 3]/2
    W = box[:, 2]
    H = box[:, 3]
    return np.stack((Xmin, Ymin, W, H), axis=1)

In [64]:
COCO_anno = []

for index, batch in enumerate(test_dataloaders):
    images = batch[0].to(device)
    filename = batch[2][0]

    bboxes, scores, class_ids = model_predict(images, model, conf_thres=0.1, iou_threshold=0.1)
    box_xywh = XcenYcenWH_to_XminYmin_WH(bboxes)
    score = scores[:, np.newaxis]
    cls_id = class_ids[:, np.newaxis]

    img_id = np.array([imageToid[filename], ] * len(cls_id))[:, np.newaxis]
    COCO_anno.append(np.concatenate((img_id, box_xywh, score, cls_id), axis=1))

    if index % 50 == 0:
        print(f"{index}/{len(test_dataloaders)} Done.")

COCO_anno = np.concatenate(COCO_anno, axis=0)
# COCO_anno.shape은 (2331,7)이 나오는데 prediction한 결과물이 2331개, 7개는 imageid, box좌표 4개, score, class id
# 이다.

0/20 Done.


In [65]:
COCO_anno

array([[ 1.00000000e+00,  5.45950546e+01,  8.31340027e+01,
         6.72633667e+01,  2.59286194e+01,  4.68564093e-01,
         1.00000000e+00],
       [ 1.00000000e+00,  2.53172150e+02,  2.72938271e+01,
         4.12192993e+01,  1.17416897e+01,  2.81199336e-01,
         1.00000000e+00],
       [ 1.00000000e+00,  2.62366028e+02,  3.81568756e+01,
         3.86082764e+01,  1.59544830e+01,  1.96980491e-01,
         1.00000000e+00],
       [ 1.00000000e+00,  1.68209839e+01,  3.07458588e+02,
         5.60678940e+01,  4.72313232e+01,  1.51311278e-01,
         1.00000000e+00],
       [ 1.00000000e+00,  1.29427597e+02,  1.20509850e+02,
         3.36961060e+01, -1.33567810e+00,  1.22508153e-01,
         1.00000000e+00],
       [ 1.00000000e+00,  1.46362259e+02,  2.67868378e+02,
         3.49928284e+01,  4.33332520e+01,  1.11201063e-01,
         1.00000000e+00],
       [ 1.00000000e+00,  3.59887848e+01,  2.61313171e+02,
         7.83865356e+00,  7.92987061e+00,  1.08486384e-01,
         0.0000000

In [66]:
cocoDt = cocoGt.loadRes(COCO_anno)

Loading and preparing results...
Converting ndarray to lists...
(57, 7)
0/57
DONE (t=0.00s)
Creating index...
index created!


In [73]:
annType = "bbox"
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.areaRng = [[0, 1e+8], [0, 20**2], [20**2, 100**2], [100**2, 1e+8]]
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()
eval_stats = cocoEval.stats

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).


IndexError: index 2 is out of bounds for axis 2 with size 2